# 顔の検出と分析

コンピュータビジョンソリューションでは、人間の顔を検出、分析、または識別できるようにするために、人工知能（AI）ソリューションを必要とすることがよくあります。例えば、小売企業のNorthwind Traders社が「スマートストア」の導入を決定したとします。このスマートストアでは、AIサービスが店舗を監視してサポートが必要な顧客を特定し、従業員にサポートを指示します。これを実現する一つの方法は、顔の検出と分析を行うことです。つまり、画像の中に顔があるかどうかを判断し、ある場合はその特徴を分析するのです。

![A robot analyzing a face](./images/face_analysis.jpg)

## Face cognitive service を使って顔を検出する

Northwind Tradersが作りたいスマートストアシステムは、顧客を検知して顔の特徴を分析できる必要があるとします。Microsoft Azureでは、Azure Cognitive Servicesの一部である**Face**を使ってこれを行うことができます。

### Cognitive Services リソースを作成する

それではまず、Azureサブスクリプションに**Cognitive Services**リソースを作成しましょう。

> **補足**: すでにCognitive Servicesリソースがある場合は、Azureポータルでその**クイックスタート**ページを開き、そのキーとエンドポイントを以下のセルにコピーするだけです。そうでない場合は、以下の手順で作成してください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
2. **&#65291;リソースの作成** ボタンをクリックし、*Cognitive Services*を検索して以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *ご自身のサプスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: S0
    - **このボックスをオンにすることにより、以下のすべてのご契約条件を読み、同意したものとみなされます**: チェックを入れます。
3. デプロイが完了するまでしばらく待ちます。次に、Cognitive Servicesリソースにアクセスし、**Overview**ページで、サービスのキーを管理するリンクをクリックします。クライアントアプリケーションからCognitive Servicesリソースに接続するには、エンドポイントとキーが必要になります。

### Cognitive Servicesリソースのキーとエンドポイントの取得

Cognitive Servicesリソースを使用するためには、クライアントアプリケーションはそのエンドポイントと認証キーを必要とします。

1. Azureポータルで、Cognitive Servicesリソースの**キーとエンドポイント**ページを開き、リソースの**キー1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。

2. リソースの**エンドポイント**をコピーし、**YOUR_COG_ENDPOINT**を置き換えて、以下のコードに貼り付けます。

3. セルの実行<span>&#9655;</span>ボタン（セルの左上）をクリックして、下のセルのコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Cognitive Servicesリソースがあるので、Faceサービスを使用してストア内の人間の顔を検出できます。

以下のコードセルを実行して、例をご覧ください。

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces)

検出された顔にはそれぞれ固有のIDが割り当てられているので、アプリケーションは検出された個々の顔を識別することができます。

下のセルを実行して、さらに何人かの店員の顔のIDを確認してみましょう。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

## 顔の属性を分析する

Faceは、単に顔を検出するだけではありません。顔の特徴や表情を分析して、年齢や感情の状態を推測することもできます。例えば、以下のコードを実行して、買い物客の顔の属性を分析してみましょう。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# Detect faces and specified facial attributes
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Display the faces and attributes (code in python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

画像に写っているお客様の感情スコアを見ると、お客様は買い物に満足されているようです。

## 類似した顔の検出

検出された顔ごとに作成された顔IDは、検出された顔を個別に識別するために使用されます。このIDを使って、検出された顔を以前に検出された顔と比較し、似た特徴を持つ顔を見つけることができます。

例えば、以下のセルを実行して、ある画像の買い物客と別の画像の買い物客を比較し、一致する顔を見つけることができます。

In [ ]:
# Get the ID of the first face in image 1
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Get the face IDs in a second image
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Find faces in image 2 that are similar to the one in image 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Show the face in image 1, and similar faces in image 2(code in python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## 顔認識

これまでに、Faceは顔と顔の特徴を検出し、互いに似ている2つの顔を識別できることがわかりました。さらに一歩進んで、特定の人の顔を認識できるようにFaceを学習させる、「顔認識」ソリューションを実装することもできます。これは、ソーシャルメディアのアプリケーションで友人の写真に自動的にタグ付けしたり、生体認証システムの一部として顔認識を使用するなど、さまざまなシナリオで役立ちます。

この仕組みを理解するために、Northwind Traders社が顔認証を利用して、IT部門の許可された従業員だけが安全なシステムにアクセスできるようにしたいと考えているとします。

まず、許可された従業員を表す**人のグループ**を作成してみます。

In [ ]:
group_id = 'employee_group_id'
try:
    # Delete group if it already exists
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

**人のグループ**ができたので、グループに入れたい従業員ごとに**人物**を追加し、各人の写真を複数枚登録することで、Faceが各人の顔の特徴を知ることができます。同じ人物でも、ポーズや表情が異なるのが理想的です。

ここでは、Wendellという一人の社員を追加し、その社員の写真を3枚登録します。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Add a person (Wendell) to the group
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Get photo's of Wendell
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Add each photo to person in person group
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Display each image
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

人物が追加され、写真が登録されると、今度はFaceに各人物を認識させるためのトレーニングを行います。

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

これでモデルが学習されたので、そのモデルを使って画像内の認識された顔を識別することができます。

In [ ]:
# Get the face IDs in a second image
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Get recognized face names
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# show recognized faces
faces.show_recognized_faces(image_path, image_faces, face_names)



## さらに学ぶ

Face cognitive serviceについてさらに学びたい場合は [Faceのドキュメント](https://docs.microsoft.com/azure/cognitive-services/face/)を参照してください。
